In [1]:
import numpy as np
import pandas as pd
import math
import sklearn.preprocessing
import datetime
from TimeBasedCV import TimeBasedCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import make_scorer, r2_score
import statsmodels.api as sm

from sklearn.linear_model import HuberRegressor
# pd.set_option('display.max_rows', None)
# more

In [2]:
df = pd.read_csv('data/factors_2000.csv', parse_dates=['DATE'])
# df = pd.read_csv('data/features_subset.csv', parse_dates=['DATE'])
# df = pd.read_csv('factors_1900.csv', parse_dates=['DATE'])
df.drop(columns=['sic2'], inplace=True)


In [3]:

#Sort observations by date and stock id
df[df.columns[2:]] = df[df.columns[2:]].astype('float32')
df = df.sort_values(by = ['DATE', 'permno'], ascending = True)
df.head()


,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,risk_premium,macro_dp,macro_ep,macro_bm,macro_ntis,macro_tbl,macro_tms,macro_dfy,macro_svar,macro_mkt-rf
0,10001,2000-01-31,20825.0000,0.062780,0.003941,0.093899,8.271964,0.025807,0.078753,-0.004188,...,-9.9418,4.423938,3.346471,0.154654,0.025359,0.0532,-0.0304,0.0057,0.005206,-4.74
1,10002,2000-01-31,94197.1875,0.487762,0.237912,0.026658,8.875427,0.050570,-0.036174,-0.123036,...,-8.0941,4.423938,3.346471,0.154654,0.025359,0.0532,-0.0304,0.0057,0.005206,-4.74
2,10009,2000-01-31,34028.2500,0.299177,0.089507,-0.093446,10.366104,0.035277,-0.036174,-0.202703,...,-6.3775,4.423938,3.346471,0.154654,0.025359,0.0532,-0.0304,0.0057,0.005206,-4.74
3,10012,2000-01-31,154794.3125,2.389731,5.710816,1.622811,13.700713,0.145356,0.914538,0.107759,...,-15.2576,4.423938,3.346471,0.154654,0.025359,0.0532,-0.0304,0.0057,0.005206,-4.74
4,10016,2000-01-31,337277.3750,0.619844,0.384206,-0.183942,11.928143,0.039299,0.325924,-0.084848,...,-15.4638,4.423938,3.346471,0.154654,0.025359,0.0532,-0.0304,0.0057,0.005206,-4.74


In [4]:
df['macro_tbl']

0          0.0532
1          0.0532
2          0.0532
3          0.0532
4          0.0532
            ...  
1649454    0.0005
1649455    0.0005
1649456    0.0005
1649457    0.0005
1649458    0.0005
Name: macro_tbl, Length: 1649459, dtype: float32

In [5]:
# df= df[~np.isnan(df['bm'])]
# df =df[~np.isnan(df['mvel1'])]

In [6]:
df['permno2'] = df['permno'].copy()
df['DATE2'] = df['DATE'].copy()
df = df.set_index(['DATE2','permno2'])

#Make a copy of  the "me" variable (market equity) before rank standartization to use afterwards for value weighting
df['mvel12'] = df['mvel1'].copy()

In [7]:
p=0.3 
df_large= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)  
df_small = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1')).reset_index(drop=True)  


C:\Users\drebi\AppData\Local\Temp\ipykernel_165412\2840026024.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_large= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)
C:\Users\drebi\AppData\Local\Temp\ipykernel_165412\2840026024.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_small = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1'

In [8]:
#Standardize all independent variables
df2 = df.copy
stdSc = StandardScaler()

features = df.columns[~df.columns.isin(['DATE', 'DATE2', 'mvel1', 'mvel12', 'permno', 'permno2', 'risk_premium'])].tolist()
df[features] = stdSc.fit_transform(df[features].astype(float))


In [9]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                #    test_period=12,
                   freq='months')

features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df[features]
y = df[['risk_premium']]


predictions = []
y_val_list = []
dates = []
r2_train = []
r2_val = []

for train_index, val_index in tscv.split(X, first_split_date= datetime.date(2006,1,31), second_split_date= datetime.date(2008,1,31)):


    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]
    lm = LinearRegression()
   
    lm.fit(X_train, y_train)
    y_pred_train = lm.predict(X_train)
    r2_train.append(r2_score(y_train, y_pred_train))
    
    y_pred_val = lm.predict(X_val)
    r2_val.append(r2_score(y_val, y_pred_val))
    predictions.append(y_pred_val)
    dates.append(y_val.index)
    y_val_list.append(y_val)

predictions_full = np.concatenate(predictions, axis=0)
y_test_list_all = np.concatenate(y_val_list, axis=0)
dates_all = np.concatenate(dates, axis=0)
print('')
r2_train_full = np.mean(r2_train)
print(f'R2 training set {r2_train_full}')
r2_val_full = np.mean(r2_val)
print(f'R2 validation set {r2_val_full}')

Train period: 2001-01-31 - 2006-01-31 ,val period: 2006-01-31 - 2008-01-31 # train records 419496 ,# val records 162353
Train period: 2002-01-31 - 2007-01-31 ,val period: 2007-01-31 - 2009-01-31 # train records 407907 ,# val records 166149
Train period: 2003-01-31 - 2008-01-31 ,val period: 2008-01-31 - 2010-01-31 # train records 403677 ,# val records 154356
Train period: 2004-01-31 - 2009-01-31 ,val period: 2009-01-31 - 2011-01-31 # train records 400348 ,# val records 132569
Train period: 2005-01-31 - 2010-01-31 ,val period: 2010-01-31 - 2012-01-31 # train records 397015 ,# val records 130189
Train period: 2006-01-31 - 2011-01-31 ,val period: 2011-01-31 - 2013-01-31 # train records 379345 ,# val records 134034
Train period: 2007-01-31 - 2012-01-31 ,val period: 2012-01-31 - 2014-01-31 # train records 366271 ,# val records 132297
Train period: 2008-01-31 - 2013-01-31 ,val period: 2013-01-31 - 2015-01-31 # train records 351026 ,# val records 139275
Train period: 2009-01-31 - 2014-01-31 ,v

In [ ]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   freq='months')


features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df_large[features]
y = df_large[['risk_premium']]

#Empty containers to save results from each window

predictions = []
y_train_list = []
y_val_list = []
dates = []
dic_r2_all = {}


for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(2007,1,31), second_split_date= datetime.date(2009,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test    = X.loc[test_index].drop('DATE', axis=1)
    y_test  = y.loc[test_index]

    lm = LinearRegression()
   
    lm.fit(X_train, y_train)
    y_pred_train = lm.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))
    
    y_pred_val = lm.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))

    y_pred_test = lm.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))


r2_val_large = np.mean(y_val_list)
print(np.mean(r2_val_large))

In [ ]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()
X = df_small[features]
y = df_small[['risk_premium']]

#Empty containers to save results from each window

predictions_top = []
y_train_list_top =[]
dates_top = []
dic_r2_all_top = {}


for train_index, val_index, test_index in tscv.split(X, first_split_date= datetime.date(2007,1,31), second_split_date= datetime.date(2009,1,31)):

    X_train = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    X_test = X.loc[test_index].drop('DATE', axis=1)
    y_test = y.loc[test_index]

    lm = LinearRegression()
   
    lm.fit(X_train, y_train)
    y_pred_train = lm.predict(X_train)
    y_train_list.append(r2_score(y_train, y_pred_train))
    
    y_pred_val = lm.predict(X_val)
    y_val_list.append(r2_score(y_val, y_pred_val))

    y_pred_test = lm.predict(X_test)
    y_test_list.append(r2_score(y_test, y_pred_test))

r2_val_small = np.mean(y_val_list)
print(np.mean(r2_val_small))

In [ ]:
chart = np.array([[r2_val_full],
                  [r2_val_large ],
                  [r2_val_small]])

r2_lm = pd.DataFrame(chart, columns=['Linear Regression'],
                     index=['Full Sample', 'Large Firms', 'Small Firms'])

r2_lm

In [ ]:
r2_lm.to_csv(r'r2_linear_model.csv')

In [ ]:
features = df.columns[~df.columns.isin(['permno','permno','DATE','DATE2','mvel12','risk_premium', 'year'])].tolist()
df['YEAR'] = df['DATE'].dt.year

X_train = df[features].loc[(df['YEAR']>=2012) & (df['YEAR']<=2016)]
y_train = df['risk_premium'].loc[(df['YEAR']>=2012) & (df['YEAR']<=2016)]

X_val = df[features].loc[(df['YEAR']>=2017) & (df['YEAR']<=2018)]
y_val = df['risk_premium'].loc[(df['YEAR']>=2017) & (df['YEAR']<=2018)]


lm_model = LinearRegression()
lm_model.fit(X_train, y_train)
y_pred_train = lm_model.predict(X_train) 

y_pred_val = lm_model.predict(X_val) 

r2_score_train = r2_score(y_train, y_pred_train)
r2_score_val = r2_score(y_val, y_pred_val)

print(f'R2 score on training set: {r2_score_train}')
print(f'R2 score on validation set: {r2_score_val}')

In [ ]:
for j in features:
    globals()['df_' + str(j)] =  df.copy()
    globals()['df_' + str(j)][str(j)] = 0

In [ ]:
dic = {}

    
for j in features:
    df_var = globals()['df_' + str(j)]
    
    X_train = df_var[features].loc[(df_var["year"]>=2012) & (df_var["year"]<=2016)]
    y_train = df_var['risk_premium'].loc[(df_var["year"]>=2012) & (df_var["year"]<=2016)]

    X_val = df_var[features].loc[(df_var["year"]>=2017) & (df_var["year"]<=2018)]
    y_val = df_var['risk_premium'].loc[(df_var["year"]>=2017) & (df_var["year"]<=2018)]
    
    lm_model = LinearRegression()
    lm_model.fit(X_train, y_train)

    y_pred_train = lm_model.predict(X_train) 

    y_pred_val = lm_model.predict(X_val) 

    r2_score_train = r2_score(y_train, y_pred_train)
    r2_score_val = r2_score(y_val, y_pred_val)



In [10]:
yhat = predictions_full.tolist()
y_true = y_test_list_all.tolist()
i = dates_all.tolist()

results = pd.DataFrame(
    {'identifier': i,
     'yhat': yhat,
     'y_true': y_true
    })


results["identifier"]= results["identifier"].astype("str")
results["date"] = results["identifier"].str[12:22]
results["id"] = results["identifier"].str[35:40]
results.drop(["identifier"],axis = 1, inplace=True)
results['date'] = pd.to_datetime(results['date'], format='%Y-%m-%d')
results['MonthYear'] = results['date'].dt.to_period('M')
results = results.sort_values(by = ['date', 'id'], ascending = True)
results = results.set_index(['MonthYear','id'])
results.head()


yhat                 y_true       date
MonthYear id                                                            
2006-01   10001    [2.351705963517218]   [-6.534900188446045] 2006-01-31
          10002  [0.49904829911069526]  [-1.3848999738693237] 2006-01-31
          10025   [1.2054678277339477]  [-0.3700000047683716] 2006-01-31
          10026   [1.0794680381948327]   [-2.434299945831299] 2006-01-31
          10028   [0.5312453262437427]   [-3.867500066757202] 2006-01-31

In [11]:
lin_results = results.reset_index()
lin_results.to_csv('results.csv', index=False)

In [12]:
results['yhat'] = results['yhat'].apply(lambda x: x[0])
results['y_true'] = results['y_true'].apply(lambda x: x[0])

In [13]:
data = df[['mvel12', 'risk_premium','macro_tbl', 'macro_svar']].copy()
data.reset_index(inplace=True)
data['permno2'] = data['permno2'].astype('str')
data['MonthYear'] = data['DATE2'].dt.to_period('M')
data.drop('DATE2', axis=1, inplace=True)
data.rename(columns={'permno2': 'id'}, inplace=True)
data.rename(columns={'mvel12': 'market_cap'}, inplace=True)
data.rename(columns={'macro_tbl': 'risk_free_rate'}, inplace=True)
data = data.set_index(['MonthYear','id'])
data

market_cap  risk_premium  risk_free_rate  macro_svar
MonthYear id                                                           
2000-01   10001  2.082500e+04     -9.941800        2.002570    0.449717
          10002  9.419719e+04     -8.094100        2.002570    0.449717
          10009  3.402825e+04     -6.377500        2.002570    0.449717
          10012  1.547943e+05    -15.257600        2.002570    0.449717
          10016  3.372774e+05    -15.463800        2.002570    0.449717
...                       ...           ...             ...         ...
2021-11   93423  3.536481e+06    -11.136800       -0.884103   -0.355373
          93426  4.296363e+05      0.654000       -0.884103   -0.355373
          93427  3.553728e+06     15.116700       -0.884103   -0.355373
          93434  1.657785e+05    -31.984699       -0.884103   -0.355373
          93436  6.109940e+07      2.711200       -0.884103   -0.355373

[1649459 rows x 4 columns]

In [14]:
bigdata = pd.merge(results, data,left_index=True, right_index=True)
bigdata.reset_index(inplace=True)
bigdata

,MonthYear,id,yhat,y_true,date,market_cap,risk_premium,risk_free_rate,macro_svar
0,2006-01,10001,2.351706,-6.5349,2006-01-31,2.842100e+04,-6.5349,1.410994,-0.428261
1,2006-01,10002,0.499048,-1.3849,2006-01-31,2.238537e+05,-1.3849,1.410994,-0.428261
2,2006-01,10025,1.205468,-0.3700,2006-01-31,2.138250e+05,-0.3700,1.410994,-0.428261
3,2006-01,10026,1.079468,-2.4343,2006-01-31,5.435421e+05,-2.4343,1.410994,-0.428261
4,2006-01,10028,0.531245,-3.8675,2006-01-31,9.776870e+03,-3.8675,1.410994,-0.428261
...,...,...,...,...,...,...,...,...,...
1975748,2021-01,93423,-3.723896,0.2333,2021-01-29,2.897716e+06,0.2333,-0.867671,-0.199544
1975749,2021-01,93426,-3.708805,1.5601,2021-01-29,3.951370e+05,1.5601,-0.867671,-0.199544
1975750,2021-01,93427,-3.811812,1.6799,2021-01-29,2.866019e+06,1.6799,-0.867671,-0.199544
1975751,2021-01,93434,-3.388458,19.3939,2021-01-29,1.001035e+05,19.3939,-0.867671,-0.199544


In [15]:
bigdata['MonthYear1'] = bigdata['MonthYear'].copy()
bigdata['MonthYear'] = bigdata['MonthYear'].astype('int64')
bigdata['NumMonth'] = bigdata['MonthYear'] - 431
bigdata['NumMonth'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [16]:
for i in bigdata['NumMonth'].unique():
    globals()['df_' + str(i)] = bigdata[bigdata['NumMonth'] == i]

In [17]:
for i in bigdata["NumMonth"].unique():
    globals()['df_' + str(i)]["rank"]= globals()['df_' + str(i)]['yhat'].rank(method='first')
    
for i in bigdata["NumMonth"].unique():
    globals()['df_' + str(i)]["DecileRank"]=pd.qcut(globals()['df_' + str(i)]['rank'].values, 10, labels = False)

#Drop normal rank, retain only decile ranks 
for i in bigdata["NumMonth"].unique():
     globals()['df_' + str(i)].drop('rank', axis=1, inplace=True)

C:\Users\drebi\AppData\Local\Temp\ipykernel_165412\3820784497.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()['df_' + str(i)]["rank"]= globals()['df_' + str(i)]['yhat'].rank(method='first')
C:\Users\drebi\AppData\Local\Temp\ipykernel_165412\3820784497.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  globals()['df_' + str(i)]["rank"]= globals()['df_' + str(i)]['yhat'].rank(method='first')
C:\Users\drebi\AppData\Local\Temp\ipykernel_165412\3820784497.py:2: SettingWithCopyWarning: 
A value is 

In [18]:
for i in bigdata["NumMonth"].unique():
    for j,g in globals()['df_' + str(i)].groupby('DecileRank'):
        globals()['df_' + str(i)+ "_" + str(j)] =  g

In [19]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)] = pd.concat([globals()['df_1_'+ str(j)], globals()['df_2_'+ str(j)]], axis=0)
    
# Generate 10 Dataframes for the 10 Decile portfolios 0-9: rank_9: top portfolio, rank_0: bottom portfolio
for i in np.arange(2,181,1):
    for j in np.arange(0,10,1):
        globals()['rank_' + str(j)] = pd.concat([globals()['rank_' + str(j)], globals()['df_' + str(i+1)+ "_" + str(j)]], axis = 0)

In [39]:
rank_1

,MonthYear,id,yhat,y_true,date,market_cap,risk_premium,risk_free_rate,macro_svar,MonthYear1,NumMonth,DecileRank,eq_weights,excess_return_stock_ew,return_stock_ew,excess_return_portfolio_ew,return_portfolio_ew,pred_excess_return_stock_ew,pred_excess_return_portfolio_ew
13,432,10092,-0.849122,5.0628,2006-01-31,1.992944e+05,5.0628,1.410994,-0.428261,2006-01,1,1,0.001493,0.007556,0.007556,4.256148,4.256148,-0.001267,-1.094050
15,432,10104,-0.998198,-1.4216,2006-01-31,6.109940e+07,-1.4216,1.410994,-0.428261,2006-01,1,1,0.001493,-0.002122,-0.002122,4.256148,4.256148,-0.001490,-1.094050
18,432,10137,-1.211850,5.5510,2006-01-31,5.154298e+06,5.5510,1.410994,-0.428261,2006-01,1,1,0.001493,0.008285,0.008285,4.256148,4.256148,-0.001809,-1.094050
19,432,10138,-0.821295,1.7386,2006-01-31,9.484766e+06,1.7386,1.410994,-0.428261,2006-01,1,1,0.001493,0.002595,0.002595,4.256148,4.256148,-0.001226,-1.094050
22,432,10147,-1.332588,-5.9853,2006-01-31,3.256122e+07,-5.9853,1.410994,-0.428261,2006-01,1,1,0.001493,-0.008933,-0.008933,4.256148,4.256148,-0.001989,-1.094050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975673,612,93149,-4.636723,-0.2998,2021-01-29,1.244703e+05,-0.2998,-0.867671,-0.199544,2021-01,181,1,0.001658,-0.000497,-0.000497,12.229275,12.229275,-0.007689,-4.867957
1975695,612,93269,-4.853381,-0.5330,2021-01-29,9.717210e+06,-0.5330,-0.867671,-0.199544,2021-01,181,1,0.001658,-0.000884,-0.000884,12.229275,12.229275,-0.008049,-4.867957
1975698,612,93285,-4.507610,9.8084,2021-01-29,1.170499e+05,9.8084,-0.867671,-0.199544,2021-01,181,1,0.001658,0.016266,0.016266,12.229275,12.229275,-0.007475,-4.867957
1975715,612,93341,-4.643777,12.2199,2021-01-29,2.416960e+05,12.2199,-0.867671,-0.199544,2021-01,181,1,0.001658,0.020265,0.020265,12.229275,12.229275,-0.007701,-4.867957


In [21]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]["eq_weights"] = 1/globals()['rank_' + str(j)].groupby('MonthYear')["id"].transform('size')

In [22]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['excess_return_stock_ew'] = globals()['rank_' + str(j)]["y_true"]*globals()['rank_' + str(j)]["eq_weights"]

In [23]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['return_stock_ew'] = globals()['rank_' + str(j)]["risk_premium"]*globals()['rank_' + str(j)]["eq_weights"]

In [24]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['excess_return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["excess_return_stock_ew"].transform('sum')

In [25]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["return_stock_ew"].transform('sum')

In [26]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['pred_excess_return_stock_ew'] = globals()['rank_' + str(j)]["yhat"]*globals()['rank_' + str(j)]["eq_weights"]

In [27]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['pred_excess_return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["pred_excess_return_stock_ew"].transform('sum')

In [28]:
for j in np.arange(0,10,1):
    globals()['montly_rank_' + str(j)] = globals()['rank_' + str(j)][["MonthYear1", "DecileRank",
                                                                      "excess_return_portfolio_ew",
                                                                      "pred_excess_return_portfolio_ew",
                                                                      "return_portfolio_ew"]]

In [29]:
for j in np.arange(0,10,1):
    globals()['montly_rank_' + str(j)]=globals()['montly_rank_' + str(j)].drop_duplicates()
    globals()['montly_rank_' + str(j)]=globals()['montly_rank_' + str(j)].set_index("MonthYear1")

In [30]:
for j in np.arange(0,10,1):
    #Time-series average of realized excess returns
    globals()["ew_mean_return_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].mean()
    #Time-series average of predicted excess returns
    globals()["ew_mean_pred_return_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["pred_excess_return_portfolio_ew"].mean()
    #Standard deviation of realized excess returns
    globals()["std_ew_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].std()
    #Annualized sharpe ratio of realized excess returns
    globals()["sharpe_ew_rank_" +  str(j)]= (globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].mean()/globals()['montly_rank_' + str(j)]["return_portfolio_ew"].std())* np.sqrt(12)

In [31]:
rank_9

,MonthYear,id,yhat,y_true,date,market_cap,risk_premium,risk_free_rate,macro_svar,MonthYear1,NumMonth,DecileRank,eq_weights,excess_return_stock_ew,return_stock_ew,excess_return_portfolio_ew,return_portfolio_ew,pred_excess_return_stock_ew,pred_excess_return_portfolio_ew
0,432,10001,2.351706,-6.534900,2006-01-31,2.842100e+04,-6.534900,1.410994,-0.428261,2006-01,1,9,0.001493,-0.009754,-0.009754,3.278418,3.278418,0.003510,2.034222
8,432,10051,2.103982,14.369100,2006-01-31,1.244038e+05,14.369100,1.410994,-0.428261,2006-01,1,9,0.001493,0.021446,0.021446,3.278418,3.278418,0.003140,2.034222
14,432,10100,1.608632,-2.536500,2006-01-31,9.609716e+03,-2.536500,1.410994,-0.428261,2006-01,1,9,0.001493,-0.003786,-0.003786,3.278418,3.278418,0.002401,2.034222
26,432,10200,1.631608,14.380000,2006-01-31,1.204560e+05,14.380000,1.410994,-0.428261,2006-01,1,9,0.001493,0.021463,0.021463,3.278418,3.278418,0.002435,2.034222
36,432,10257,1.550062,33.130001,2006-01-31,2.313360e+04,33.130001,1.410994,-0.428261,2006-01,1,9,0.001493,0.049448,0.049448,3.278418,3.278418,0.002314,2.034222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975705,612,93304,-1.672786,-5.235400,2021-01-29,1.644458e+06,-5.235400,-0.867671,-0.199544,2021-01,181,9,0.001656,-0.008668,-0.008668,-0.001075,-0.001075,-0.002770,-1.753695
1975711,612,93330,-1.849182,3.957000,2021-01-29,5.263850e+06,3.957000,-0.867671,-0.199544,2021-01,181,9,0.001656,0.006551,0.006551,-0.001075,-0.001075,-0.003062,-1.753695
1975733,612,93368,-1.904506,0.741100,2021-01-29,1.437655e+05,0.741100,-0.867671,-0.199544,2021-01,181,9,0.001656,0.001227,0.001227,-0.001075,-0.001075,-0.003153,-1.753695
1975734,612,93369,-1.575554,4.977500,2021-01-29,2.909879e+06,4.977500,-0.867671,-0.199544,2021-01,181,9,0.001656,0.008241,0.008241,-0.001075,-0.001075,-0.002609,-1.753695


In [32]:
# For the zero-net-investment long-short portfolio the top (long) and bottom(short) decile portfolios are needed

long_monthly = rank_9[["NumMonth","MonthYear1",
                       "DecileRank",
                       "excess_return_portfolio_ew",
                       "pred_excess_return_portfolio_ew",
                       "return_portfolio_ew",
                        ]].drop_duplicates()

short_monthly = rank_0[["NumMonth",
                        "MonthYear1",
                        "DecileRank",
                        "excess_return_portfolio_ew",
                        "pred_excess_return_portfolio_ew",
                        "return_portfolio_ew",
                        ]].drop_duplicates()

# Create a column, indication the stategy 
long_monthly["Strategy"]= "long"
short_monthly["Strategy"]= "short"

In [33]:
# Merge to get the zero net investment portfolio
zeronet_monthly= pd.concat([long_monthly, short_monthly])
zeronet_monthly = zeronet_monthly.sort_values(by = ['NumMonth',"Strategy"])

In [34]:

#Create two new columns containing the exess return of the portfolio and initially set the values to zero.
zeronet_monthly["excess_return_zeronet_ew"] =0


In [35]:

# excess return zeronet in t = (weigted excess return long in t) - (weigted excess return short in t)
for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,7] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 3]-zeronet_monthly.iloc[i+1, 3]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 3]-zeronet_monthly.iloc[i, 3]
    

C:\Users\drebi\AppData\Local\Temp\ipykernel_165412\2078608670.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '20.21989707890434' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 3]-zeronet_monthly.iloc[i, 3]


In [36]:
#Create two new columns containing predicted the exess return of the portfolio and initially set the values to zero.
zeronet_monthly["pred_excess_return_zeronet_ew"] =0

In [37]:
# predicted excess return zeronet in t = (weigted predicted excess return long in t) - (weigted predicted excess return short in t)
for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,7] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 4]-zeronet_monthly.iloc[i+1, 4]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 4]-zeronet_monthly.iloc[i, 4]

C:\Users\drebi\AppData\Local\Temp\ipykernel_165412\93502614.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-8.699640535595506' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 4]-zeronet_monthly.iloc[i, 4]


In [38]:
zeronet_monthly

,NumMonth,MonthYear1,DecileRank,excess_return_portfolio_ew,pred_excess_return_portfolio_ew,return_portfolio_ew,Strategy,excess_return_zeronet_ew,pred_excess_return_zeronet_ew
0,1,2006-01,9,3.278418,2.034222,3.278418,long,20.219897,-8.699641
11,1,2006-01,0,5.511885,-2.284310,5.511885,short,-2.233467,4.318532
6699,2,2006-02,9,-2.938582,-1.639981,-2.938582,long,8.450468,-0.644329
6705,2,2006-02,0,-5.521639,-6.097807,-5.521639,short,2.583057,4.457826
13379,3,2006-03,9,-0.014847,-1.095010,-0.014847,long,-5.506792,-5.002797
...,...,...,...,...,...,...,...,...,...
1957841,179,2020-11,0,20.651438,4.881781,20.651438,short,-6.895014,4.549717
1963750,180,2020-12,9,6.187834,2.408292,6.187834,long,14.463604,2.473488
1963762,180,2020-12,0,3.841840,-2.206042,3.841840,short,2.345994,4.614334
1969728,181,2021-01,9,-0.001075,-1.753695,-0.001075,long,3.842915,-0.452347


In [ ]:
#Create two new columns containing return of the portfolio and initially set the values to zero.       
zeronet_monthly["return_zeronet_ew"] =0

In [ ]:
# return zeronet in t = (weigted return long in t) - (weigted return short in t)
for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,7] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 5]-zeronet_monthly.iloc[i+1, 5]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 5]-zeronet_monthly.iloc[i, 5]
        

In [ ]:
zeronet_monthly.iloc[:, 5]

In [ ]:
zeronet_monthly = zeronet_monthly[['NumMonth',
                                   'MonthYear1', 
                                   'excess_return_zeronet_ew',
                                   'pred_excess_return_zeronet_ew',
                                   'return_zeronet_ew',
                                   ]].drop_duplicates()

zeronet_monthly

In [ ]:
ew_mean_return_zeronet= zeronet_monthly["excess_return_zeronet_ew"].mean()
#Time-series average of predicted excess returns
ew_mean_pred_return_zeronet = zeronet_monthly["pred_excess_return_zeronet_ew"].mean()
#Standard deviation of realized excess returns
std_ew_zeronet = zeronet_monthly["excess_return_zeronet_ew"].std()
#Annualized sharpe ratio of realized excess returns
sharpe_ew_zeronet = (zeronet_monthly["excess_return_zeronet_ew"].mean()/zeronet_monthly["return_zeronet_ew"].std())* np.sqrt(12)

In [ ]:
chart_np = np.array([[ew_mean_pred_return_rank_0, ew_mean_return_rank_0, std_ew_rank_0, sharpe_ew_rank_0],
                     [ew_mean_pred_return_rank_1, ew_mean_return_rank_1, std_ew_rank_1, sharpe_ew_rank_1],
                     [ew_mean_pred_return_rank_2, ew_mean_return_rank_2, std_ew_rank_2, sharpe_ew_rank_2],
                     [ew_mean_pred_return_rank_3, ew_mean_return_rank_3, std_ew_rank_3, sharpe_ew_rank_3],
                     [ew_mean_pred_return_rank_4, ew_mean_return_rank_4, std_ew_rank_4, sharpe_ew_rank_4],
                     [ew_mean_pred_return_rank_5, ew_mean_return_rank_5, std_ew_rank_5, sharpe_ew_rank_5],
                     [ew_mean_pred_return_rank_6, ew_mean_return_rank_6, std_ew_rank_6, sharpe_ew_rank_6],
                     [ew_mean_pred_return_rank_7, ew_mean_return_rank_7, std_ew_rank_7, sharpe_ew_rank_7],
                     [ew_mean_pred_return_rank_8, ew_mean_return_rank_8, std_ew_rank_8, sharpe_ew_rank_8],
                     [ew_mean_pred_return_rank_9, ew_mean_return_rank_9, std_ew_rank_9, sharpe_ew_rank_9],
                     [ew_mean_pred_return_zeronet, ew_mean_return_zeronet, std_ew_zeronet, sharpe_ew_zeronet]])

ew_df = pd.DataFrame(chart_np, columns=['Pred', 'Real', 'Std', 'Sharpe'],
                              index=['Low (L)', '2', '3', '4', '5','6','7','8',"9",'High (H)', "H-L"])

ew_df['Pred'] = pd.Series(["{0:.2f}%".format(val * 100) for val in ew_df['Pred']], index = ew_df.index)
ew_df['Real'] = pd.Series(["{0:.2f}%".format(val * 100) for val in ew_df['Real']], index = ew_df.index)
ew_df['Std'] = pd.Series(["{0:.2f}%".format(val * 100) for val in ew_df['Std']], index = ew_df.index)
ew_df['Sharpe'] = pd.Series([("%.2f" % round(val, 2)) for val in ew_df['Sharpe']], index = ew_df.index)
ew_df